In [1]:
import numpy as np
import re
import pickle
import h5py
from random import sample
import tensorflow as tf
import tensorflow_probability as tfp
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.layers import Input
from keras.layers import Concatenate
from keras.layers import LSTM, Embedding
from tensorflow.keras import layers
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras import backend as K
from feature_funcs import *
from keras.utils import to_categorical

Using TensorFlow backend.


## Prepare Training Data

In [2]:
# Create dictionary
with open("../data/Jigs.txt") as my_file:
    abc_text = my_file.read()

# Cut out unnecessary backslashes
abc_text = re.sub('\\\\+\n', '\n', abc_text)

# Find starting index of the data we care about
start_ind = abc_text.find("X:")
abc_text = abc_text[start_ind:]

# Encode data
num_to_char, char_to_num = create_dictionaries(abc_text)
vocab_length = len(num_to_char)

### Separate Songs

In [ ]:
songs = re.findall(r'(T:(?:.+\n)+\n+)X:', abc_text)

song_texts = []

for song in songs:
    song_text = re.search(r'(P:.+|K:.+)', song, re.DOTALL)
    song_texts.append(song_text[0])
    
# Get aggregate statistics about song texts
print("Min text length: ", min([len(text) for text in song_texts]))
print("Max text length: ", max([len(text) for text in song_texts]))
print("Average text length: ", np.mean([len(text) for text in song_texts]))

# Make sure all song texts are the same length
song_texts = [text[:170] for text in song_texts]

### Extract Metadata elements

In [ ]:
# Extract titles
titles = []

for song in songs:
    title = re.search(r'(T:.+\n% Nottingham Music Database\n)', song).group(1)
    titles.append(title)
    
# Pad titles to same length
max_length = max(len(title) for title in titles)
titles = [title.ljust(max_length) for title in titles]


# Extract authors
authors = []

for song in songs:
    author = re.search(r'\n(S:.+\n)', song).group(1)
    authors.append(author)
    
# Pad authors to same length
max_length = max(len(author) for author in authors)
authors = [author.ljust(max_length) for author in authors]


# Extract meters
meters = []

for song in songs:
    meter = re.search(r'(M:\d/\d\n)', song).group(1)
    meters.append(meter)

    
# Pad meters to same length
max_length = max(len(meter) for meter in meters)
meters = [meter.ljust(max_length) for meter in meters]


### Encode data to numeric format

In [ ]:
text_nums_list = encoder2(song_texts, char_to_num)

title_nums = encoder2(titles, char_to_num)
author_nums = encoder2(authors, char_to_num)
meter_nums = encoder2(meters, char_to_num)

In [ ]:
meter_nums

### Option 2: Use Pickled data where songs are not separated (data already encoded to numeric)

In [ ]:
# Open pickled training data so you don't have to re-run create_training
x_file_pickle = open('../data/x_train_pickle.obj', 'rb')
y_file_pickle = open('../data/y_train_pickle.obj', 'rb')

x_train = pickle.load(x_file_pickle)
y_train = pickle.load(y_file_pickle)

x_file_pickle.close()
y_file_pickle.close()

vocab_length = x_train.shape[2]
vocab_length

In [ ]:
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)
print(vocab_length)

## Build & Compile LSTM (Model 1)

Sources: <br />
https://towardsdatascience.com/how-to-generate-music-using-a-lstm-neural-network-in-keras-68786834d4c5  <br />
https://github.com/aamini/introtodeeplearning/blob/master/lab1/Part2_Music_Generation.ipynb  <br /> https://medium.com/datadriveninvestor/music-generation-using-deep-learning-85010fb982e2 
<br /> https://keras.io/examples/lstm_text_generation/

In [ ]:
model = tf.keras.Sequential()
model.add(layers.LSTM(128, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
model.add(layers.LSTM(256, return_sequences=True))
model.add(layers.LSTM(512))
model.add(layers.Dense(vocab_length, activation='softmax'))

model.summary()


In [ ]:
# rmsprop: Divide the learning rate for a weight by a running average of the
# magnitues of the recent gradients for that weight
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

### Train RNN
Epoch: When the Neural Network sees all of the training data <br />
Batch: Subset of the data <br />
i.e. If you have 1000 data points, your batch size is 500 and you want 1 epoch, then the NN will do 2 iterations.

In [ ]:
# Use checkpoints to save training weights before the model finishes training
# Using this file path, the model checkpoints will be saved with the epoch number 
# and the validation loss in the filename.
weights_filepath = "weights.hdf5"

checkpoint = ModelCheckpoint(
    weights_filepath, monitor="loss", verbose=0,
    save_best_only=True, mode="min")

callbacks_list = [checkpoint]

# Fit model
model.fit(x_train, y_train, epochs=1, batch_size=400, callbacks=callbacks_list)

In [ ]:
# Continue training model
model.load_weights("weights.hdf5")
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

# Fit model
model.fit(x_train, y_train, epochs=1, batch_size=200, callbacks=callbacks_list)

In [ ]:
# Open pickled test data so you don't have to re-run create_training
x_file_pickle = open('../data/x_test_pickle.obj', 'rb')
y_file_pickle = open('../data/y_test_pickle.obj', 'rb')

x_test = pickle.load(x_file_pickle)
y_test = pickle.load(y_file_pickle)

x_file_pickle.close()
y_file_pickle.close()

In [ ]:
predictions = model.predict(x_test)
predictions

In [ ]:
text_predictions = decoder(predictions, num_to_char)
print(text_predictions)

## Build & Compile LSTM (Model 2) 

Sources:<br /> https://benjamintseng.com/portfolio/nlp-pubmed-data-using-tensorflow-and-keras/ <br />
https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

In [ ]:
def LSTM_Builder1(song_text_nums, metadata_nums, str_length, vocab_size):
    # Prepare data
    if str_length+1 > len(song_text_nums[0]):
        return "Your string length is too long for the data."

    # The x_values begin at the start of each song text and are str_length characters long;
    # Concatenate these with the metadata at the beginning
    # The y_values are one character after the end of the x_values
    x_data = np.concatenate((metadata_nums[0], song_text_nums[0][0:str_length]))
    y_data = [song_text_nums[0][str_length]]
    
    for ind, song in enumerate(song_text_nums[1:], start=1):
        x_concat = np.concatenate((metadata_nums[i], np.array(song[0:str_length])))
        x_data = np.vstack((x_data, x_concat))
        y_data.append(song[str_length])

    # Convert x and y data to tensors
    x_train = to_categorical(x_data, num_classes=vocab_size)
    y_train = to_categorical(y_data, num_classes=vocab_size)
    
    # Build NN
    model = tf.keras.Sequential()
    model.add(layers.LSTM(128, input_shape=(x_train.shape[1], x_train.shape[2]), return_sequences=True))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(vocab_length, activation='softmax'))
    
    return x_train, y_train, model


In [ ]:
x_train, y_train, mod1 = LSTM_Builder1(text_nums_list[:train_ind], meter_nums, 160, vocab_length)
mod1.summary()

In [ ]:
# rmsprop: Divide the learning rate for a weight by a running average of the
# magnitues of the recent gradients for that weight
mod1.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
# Use checkpoints to save training weights before the model finishes training
# Using this file path, the model checkpoints will be saved with the epoch number 
# and the validation loss in the filename.
weights_filepath = "mod1_weights.hdf5"

checkpoint = ModelCheckpoint(
    weights_filepath, monitor="loss", verbose=0,
    save_best_only=True, mode="min")

callbacks_list = [checkpoint]

In [ ]:
# Fit model
mod1.fit(x_train, y_train, epochs=1000, batch_size=100, callbacks=callbacks_list)

In [ ]:
# Get testing data
test_nums = text_nums_list[train_ind:]
meter_test = meter_nums[train_ind:]
str_length = 160

x_test = np.concatenate((meter_test[0], test_nums[0][0:str_length]))
y_test = [test_nums[0][str_length]]

# The x_values begin at the start of each song text and are str_length characters long;
# Concatenate these with the metadata at the beginning
# The y_values are one character after the end of the x_values    
for ind, song in enumerate(test_nums[1:], start=1):
    x_concat = np.concatenate((meter_test[ind], song[0:str_length]))
    x_test = np.vstack((x_test, x_concat))
    y_test.append(song[str_length])

# Convert x and y data to tensors
x_test = to_categorical(x_test, num_classes=vocab_length)
y_test = to_categorical(y_test, num_classes=vocab_length)

In [ ]:
x_test.shape, y_test.shape

### Generate Predictions

In [ ]:
def decoder(binary_matrix, dictionary):
    '''Convert numeric list a text string.'''
    text_list = []
    
    for row in binary_matrix:
        max_ind = np.argmax(row)
        text_list.append(dictionary[max_ind])
    
    return "".join(text_list)

In [ ]:
def decoder2(pred_matrix, dictionary):
    '''Convert numeric list a text string.'''
    my_session = tf.Session()
    
    text_list = []
    
    samples = tf.random.categorical(pred_matrix, num_samples=1)
    samples = tf.squeeze(samples, axis=-1)
    vals = sess.run(samples)
    
    text_pred = "".join([dictionary[i] for i in vals])
        
    return text_pred


In [ ]:
decoder1(predictions, num_to_char)

In [ ]:
starter = x_test[0]
len_starter = len(x_test[0])

x_vals = np.reshape(starter, (1, starter.shape[0], starter.shape[1]))
result = decoder(starter, num_to_char)
seq_len = 200

for i in range(1,seq_len):
    prediction = mod2.predict(x_vals)
    starter = np.vstack((starter, prediction))
    starter = starter[1:1+len_starter,]
    
    x_vals = np.reshape(starter, (1, starter.shape[0], starter.shape[1]))
    
    text_prediction = decoder2(prediction, num_to_char)
    result += text_prediction

In [ ]:
sampled_indices = tf.random.categorical(predictions, num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1)
sess.run(sampled_indices)

In [ ]:
dist = tfp.distributions.OneHotCategorical(probs=prediction)
sample = tfp.distributions.Sample(dist)
sample
sess = tf.Session()
vals = sess.run(sampled_indices)

In [ ]:
print(result)

## Build & Compile LSTM (Model 3; using music21 attributes) 

In [3]:
from music21 import *
import numpy as np

import time
import math

In [4]:
f = open("../../Anis/data/jiggs.txt", "r")
raw_input = f.read()

In [5]:
class Repertoir():
    def __init__(self, path):
        self.path = path
        f = open(path, "r")
        self.string = f.read()
        self.handler = abcFormat.ABCHandler()
        self.handler.process(self.string)
        self.songs_handlers = self.handler.splitByReferenceNumber()
        self.songs = {}
        self.__process()
    
    def __str__(self):
        return self.string
    
    
    def __process(self):
        for ref_number, handler in self.songs_handlers.items():
            self.songs[ref_number] = Song(handler)
            
    def get_part_vocab(self):
        tokens = []
        for ref_number, song in self.songs.items():
            tokens+= song.part
        tokens = list(set(tokens))            
        return tokens
    
    def get_metadata_vocab(self, key):
        tokens = []
        for ref_number, song in self.songs.items():
            tokens+= [song.metadata[key]]
        tokens = list(set(tokens))            
        return tokens    

In [6]:
class Song():
    def __init__(self, handler):
        self.handler = handler
        self.metadata = {
            'X':1,
            'T':'Unknown',
            'S':'Unknown',
            'M':'none',
            'L':'',
            'Q':'',
            'K':''
        }
        self.part = []
        self.__process()
        
    def __process(self):
        for token in self.handler.tokens:
            meta_data_ended=False
            if isinstance(token, abcFormat.ABCMetadata):
                if token.tag in self.metadata.keys():
                    if self.metadata[token.tag]=='' or not meta_data_ended:
                        self.metadata[token.tag] = token.data
                else:
                    self.metadata[token.tag] = token.data
            elif isinstance(token, abcFormat.ABCNote ) or isinstance(token, abcFormat.ABCBar):
                meta_data_ended = True
                self.part.append(token.src)
    
    def __str__(self):
        return self.to_abc()
    
    def to_abc(self):
        output = ''
        for key, value in self.metadata.items():
            output+= key+':'+value+"\n"
        for note in self.part:
            output+=note
        return output

In [ ]:
def generate_char_idx_mappings(vocab):
    char2idx = {u:i for i, u in enumerate(vocab)}
    idx2char = np.array(vocab)
    return char2idx, idx2char

def get_input_tensors(part, k, m, part_char2idx, k_char2idx, m_char2idx):
    part_tensor = torch.tensor([part_char2idx[note] for note in part[0:-1]], dtype=torch.long)
    k_tensor = torch.tensor([k_char2idx[k] for note in part[0:-1]], dtype=torch.long)
    m_tensor = torch.tensor([m_char2idx[m] for note in part[0:-1]], dtype=torch.long)
    return part_tensor, k_tensor, m_tensor,

def get_target_tensor(part, part_char2idx):
    target_tensor = torch.tensor([part_char2idx[note] for note in part[1:]], dtype=torch.long)
    return target_tensor

In [7]:
rep = Repertoir('../../Anis/data/jiggs.txt')
print(str(rep.songs[1]))

X:1
T:A and D
S:EF
M:6/8
L:
Q:
K:D
P:B
f|"A"eccc2f|"A"eccc2f|"A"eccc2f|"Bm"BcB"E7"B2f|"A"eccc2f|"A"eccc2c/2d/2|"D"efe"E7"dcB|[1"A"Acea2:|[2"A"Aceag=g||"D"f2fFdd|"D"AFAf2e/2f/2|"G"g2gecd|"Em"efd"A7"cBA|"D"f^efdcd|"D"AFAf=ef|"G"gfg"A7"ABc|[1"D"d3d2e:|[2"D"d3d2||


In [ ]:
part_vocab = rep.get_part_vocab()
m_vocab = rep.get_metadata_vocab('M')
k_vocab = rep.get_metadata_vocab('K')

part_vocab_size = len(part_vocab)
k_vocab_size = len(k_vocab)
m_vocab_size = len(m_vocab)

In [ ]:
part_char2idx, part_idx2char = generate_char_idx_mappings(part_vocab)
k_char2idx, k_idx2char = generate_char_idx_mappings(k_vocab)
m_char2idx, m_idx2char = generate_char_idx_mappings(m_vocab)

### Create Training Set

In [ ]:
part_len = 60
num_songs = int(0.9*len(rep.songs))
k_len = min([len(rep.songs[i].metadata['K']) for i in range(1,num_songs+1)])
m_len = min([len(rep.songs[i].metadata['M']) for i in range(1,num_songs+1)])

part_num_matrix = np.zeros((num_songs, part_len))
y_vals = []
k_num_matrix = np.zeros((num_songs, 1))
m_num_matrix = np.zeros((num_songs, m_len))

for i in range(1,num_songs+1):
    song = rep.songs[i]
    part_num_matrix[i-1,] = np.array([part_char2idx[song.part[0:part_len][j]] for j in range(len(song.part[0:part_len]))])
    k_num_matrix[i-1,0] = [k_char2idx[song.metadata['K']]][0]
    m_num_matrix[i-1,] = [m_char2idx[song.metadata['M']]]
    y_vals.append(part_char2idx[song.part[part_len]])

# Convert y_vals to one_hot_encoded vectors
y_data = to_categorical(y_vals, num_classes=part_vocab_size)

### Build LSTM NN

In [ ]:
def LSTM_Builder2(lstm_dim, dropout_pct, batch_size,
                  embedding_dim, seq_len,
                  part_voc_size, k_voc_size, m_voc_size, 
                  part_num_matrix, k_num_matrix, m_num_matrix):
    
    voc_input = Input(shape=(part_num_matrix.shape[1],))
    k_input = Input(shape=(k_num_matrix.shape[1],))
    m_input = Input(shape=(m_num_matrix.shape[1],))
    
    voc_embedding = Embedding(part_voc_size, embedding_dim)(voc_input)
    
    lstm_out = LSTM(lstm_dim, dropout=dropout_pct)(voc_embedding)
    
    concat = Concatenate()([lstm_out, k_input, m_input])

    output = Dense(part_voc_size, activation='softmax')(concat)
    
    model = Model(inputs=[voc_input, k_input, m_input], outputs=output)
    
    return model


In [ ]:
mod2 = LSTM_Builder2(256, 0.2, 100, 
                     128, 60,
                     part_vocab_size, k_vocab_size, m_vocab_size,
                     part_num_matrix, 
                     k_num_matrix, 
                     m_num_matrix)

mod2.summary()

Reference: https://benjamintseng.com/portfolio/nlp-pubmed-data-using-tensorflow-and-keras/

In [ ]:
def LSTM_Builder3(lstm_dims, dropout_pct, batch_size,
                  embedding_dim, seq_len,
                  part_voc_size, k_voc_size, m_voc_size, 
                  part_num_matrix, k_num_matrix, m_num_matrix):
    
    voc_input = Input(shape=(part_num_matrix.shape[1],))
    k_input = Input(shape=(k_num_matrix.shape[1],))
    m_input = Input(shape=(m_num_matrix.shape[1],))
    
    voc_embedding = Embedding(part_voc_size, embedding_dim)(voc_input)
    k_embedding = Embedding(k_voc_size, embedding_dim)(k_input)
    m_embedding = Embedding(m_voc_size, embedding_dim)(m_input)
    
    concat = Concatenate(axis=1)([m_embedding, k_embedding, voc_embedding])
    
    lstm1 = LSTM(lstm_dims[0], dropout=dropout_pct, return_sequences=True)(concat)
    lstm2 = LSTM(lstm_dims[1], dropout=dropout_pct, return_sequences=True)(lstm1)
    lstm3 = LSTM(lstm_dims[2], dropout=dropout_pct)(lstm2)

    output = Dense(part_voc_size, activation='softmax')(lstm3)
    
    model = Model(inputs=[voc_input, k_input, m_input], outputs=output)
    
    return model


In [ ]:
mod3 = LSTM_Builder3([256, 512, 256], 0.2, 100, 
                     256, 60,
                     part_vocab_size, k_vocab_size, m_vocab_size,
                     part_num_matrix, 
                     k_num_matrix, 
                     m_num_matrix)

mod3.summary()

In [ ]:
mod3.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# Use checkpoints to save training weights before the model finishes training
# Using this file path, the model checkpoints will be saved with the epoch number 
# and the validation loss in the filename.
weights_filepath = "mod3_weights.hdf5"

checkpoint = ModelCheckpoint(
    weights_filepath, monitor="loss", verbose=0,
    save_best_only=True, mode="min")

callbacks_list = [checkpoint]

In [ ]:
mod3.fit([part_num_matrix, k_num_matrix, m_num_matrix],
         y_data, 
         epochs=5000, batch_size=50, shuffle=True, callbacks=callbacks_list)

In [ ]:
part_num_matrix.shape, k_num_matrix.shape, m_num_matrix.shape

In [ ]:
mod3.predict([part_num_matrix, k_num_matrix, m_num_matrix]).shape

### Get Predictions

In [ ]:
# Set up testing data
my_session = tf.Session()
mod3.reset_states()

starter = rep.songs[num_songs]
start_ind = int(0.9*len(rep.songs))
end_ind = len(rep.songs)
seq_len = 60

input_part = np.zeros((1, part_len))
sequences = input_part
input_k = np.zeros((1, 1))
input_m = np.zeros((1, m_len))

for i in range(start_ind,start_ind+1):
    song = rep.songs[i]
    input_part[i-start_ind,] = np.array([part_char2idx[song.part[0:part_len][j]] for j in range(len(song.part[0:part_len]))])
    input_k[i-start_ind,0] = [k_char2idx[song.metadata['K']]][0]
    input_m[i-start_ind,] = [m_char2idx[song.metadata['M']]]

# Starting string:
beginning = str(rep.songs[start_ind])
result = []


# Generate predictions
for i in range(seq_len):
    prediction = mod3.predict([input_part, input_k, input_m])
    
    # Sample from output of probabilities
    sample = tf.random.categorical(prediction, num_samples=1)
    sample = tf.squeeze(sample, axis=-1)
    numeric_pred = my_session.run(sample)[0]
    
    # Pass the prediction, along with the hidden state, back to the model
    sequences = np.append(sequences, numeric_pred)
    input_part = sequences[i:part_len+i].reshape(1, part_len)
    
    # Text predictions
    text_val = part_idx2char[numeric_pred]
    result.append(text_val)

result = "".join(result)
print(beginning)
print("______")
print(result)



## Model 4: music21 attributes in single character encoding)

### Create training data

In [141]:
jig_rep = Repertoir('../data/Jigs.txt')

num_to_char, char_to_num = create_dictionaries(str(jig_rep))

In [142]:
vocab_length = len(num_to_char)

num_songs = int(0.9*len(jig_rep.songs))
total_songs = len(jig_rep.songs)

part_len = min(len(list("".join(jig_rep.songs[i].part))) for i in range(1,total_songs+1)) - 2
k_len = min([len(list(jig_rep.songs[i].metadata['K'])) for i in range(1,total_songs+1)])
m_len = min([len(list(jig_rep.songs[i].metadata['M'])) for i in range(1,total_songs+1)])

part_num_matrix = np.zeros((num_songs, part_len))
y_vals = []
k_num_matrix = np.zeros((num_songs, k_len))
m_num_matrix = np.zeros((num_songs, m_len))

for i in range(1,num_songs+1):
    song = jig_rep.songs[i]
    part_string = "".join(song.part)
    part_num_matrix[i-1,] = np.array([char_to_num[part_string[0:part_len][j]] for j in range(part_len)])
    
    k_string = "".join(song.metadata['K'])
    k_num_matrix[i-1,0] = np.array([char_to_num[k_string[j]] for j in range(k_len)])
    
    m_string = "".join(song.metadata['M'])
    m_num_matrix[i-1,] = np.array([char_to_num[m_string[j]] for j in range(m_len)])
    
    y_vals.append(char_to_num[part_string[part_len]])

# Convert y_vals to one_hot_encoded vectors
y_data = to_categorical(y_vals, num_classes=vocab_length)

In [143]:
part_num_matrix.shape, k_num_matrix.shape, m_num_matrix.shape

((306, 150), (306, 1), (306, 3))

In [144]:
def LSTM_Builder4(lstm_dims, dropout_pct, batch_size,
                  embedding_dim,
                  vocab_size, 
                  part_num_matrix, k_num_matrix, m_num_matrix):
    
    voc_input = Input(shape=(part_num_matrix.shape[1],))
    k_input = Input(shape=(k_num_matrix.shape[1],))
    m_input = Input(shape=(m_num_matrix.shape[1],))
    
    voc_embedding = Embedding(vocab_size, embedding_dim)(voc_input)
    k_embedding = Embedding(vocab_size, embedding_dim)(k_input)
    m_embedding = Embedding(vocab_size, embedding_dim)(m_input)
    
    concat = Concatenate(axis=1)([m_embedding, k_embedding, voc_embedding])
    
    lstm1 = LSTM(lstm_dims[0], dropout=dropout_pct, return_sequences=True)(concat)
    lstm2 = LSTM(lstm_dims[1], dropout=dropout_pct, return_sequences=True)(lstm1)
    lstm3 = LSTM(lstm_dims[2], dropout=dropout_pct)(lstm2)

    output = Dense(vocab_size, activation='softmax')(lstm3)
    
    model = Model(inputs=[voc_input, k_input, m_input], outputs=output)
    
    return model


In [145]:
vocab_length

87

In [146]:
mod4 = LSTM_Builder4([200, 336, 200], 0.2, 50, 
                     256,vocab_length,
                     part_num_matrix, k_num_matrix, m_num_matrix)

mod4.summary()

Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 3)            0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_19 (InputLayer)           (None, 150)          0                                            
__________________________________________________________________________________________________
embedding_21 (Embedding)        (None, 3, 256)       22272       input_21[0][0]                   
____________________________________________________________________________________________

In [147]:
mod4.compile(loss='categorical_crossentropy', optimizer='adam')

In [148]:
# Use checkpoints to save training weights before the model finishes training
weights_filepath = "mod4_weights.hdf5"

checkpoint = ModelCheckpoint(
    weights_filepath, monitor="loss", verbose=0,
    save_best_only=True, mode="min")

callbacks_list = [checkpoint]

In [ ]:
mod4.fit([part_num_matrix, k_num_matrix, m_num_matrix],
         y_data, 
         epochs=5000, batch_size=50, shuffle=True, callbacks=callbacks_list)

Epoch 1/5000
306/306 [==============================] - 11s 37ms/step - loss: 4.1881
Epoch 2/5000
306/306 [==============================] - 8s 27ms/step - loss: 3.1653
Epoch 3/5000
306/306 [==============================] - 10s 33ms/step - loss: 2.9420
Epoch 4/5000
306/306 [==============================] - 10s 34ms/step - loss: 2.8932
Epoch 5/5000
306/306 [==============================] - 11s 35ms/step - loss: 2.8719
Epoch 6/5000
306/306 [==============================] - 12s 39ms/step - loss: 2.8420
Epoch 7/5000
 50/306 [===>..........................] - ETA: 10s - loss: 2.7638

### Make Predictions

In [ ]:
# Set up testing data
my_session = tf.Session()
mod4.reset_states()

starter = jig_rep.songs[num_songs]
start_ind = num_songs
end_ind = len(jig_rep.songs)
seq_len = 100

input_part = np.zeros((1, part_len))
sequences = input_part
input_k = np.zeros((1, k_len))
input_m = np.zeros((1, m_len))

for i in range(start_ind,start_ind+1):
    song = jig_rep.songs[i]
    
    part_string = "".join(song.part)
    k_string = "".join(song.metadata['K'])
    m_string = "".join(song.metadata['M'])
    
    input_part[i-start_ind,] = np.array([char_to_num[part_string[0:part_len][j]] for j in range(part_len)])
    input_k[i-start_ind,] = np.array([char_to_num[k_string[j]] for j in range(k_len)])
    input_m[i-start_ind,] = np.array([char_to_num[m_string[j]] for j in range(m_len)])

# Starting string:
beginning = str(jig_rep.songs[start_ind])
result = []


# Generate predictions
for i in range(seq_len):
    prediction = mod4.predict([input_part, input_k, input_m])
    
    # Sample from output of probabilities
    sample = tf.random.categorical(prediction, num_samples=1)
    sample = tf.squeeze(sample, axis=-1)
    numeric_pred = my_session.run(sample)[0]
    
    # Pass the prediction, along with the hidden state, back to the model
    sequences = np.append(sequences, numeric_pred)
    input_part = sequences[i:part_len+i].reshape(1, part_len)
    
    # Text predictions
    text_val = num_to_char[numeric_pred]
    result.append(text_val)

result = "".join(result)
print(beginning)
print("______")
print(result)



## Model 5: Train Model on Multiple Repertoires

In [ ]:
ashover_rep = Repertoir('../data/Ashover.txt')
carols_rep = Repertoir('../data/Carols.txt')
waltzes_rep = Repertoir('../data/Waltzes.txt')
slip_jigs_rep = Repertoir('../data/Slip Jigs.txt')
reels_uz_rep = Repertoir('../data/Reels U-Z.txt')
reels_rt_rep = Repertoir('../data/Reels R-T.txt')
reels_mq_rep = Repertoir('../data/Reels M-Q.txt')
#reels_hl_rep = Repertoir('../data/Reels H-L.txt') # There is somthing wrong with these files
#reels_dg_rep = Repertoir('../data/Reels D-G.txt') # Ignoring
reels_ac_rep = Repertoir('../data/Reels A-C.txt')
playford_rep = Repertoir('../data/Playford.txt')
morris_rep = Repertoir('../data/Morris.txt')
jigs_rep = Repertoir('../data/Jigs.txt')
hornpipes_rep = Repertoir('../data/Hornpipes.txt')

In [ ]:
rep_list = [ashover_rep, carols_rep, waltzes_rep, slip_jigs_rep,
           reels_uz_rep, reels_rt_rep, reels_mq_rep, #reels_hl_rep,
           #reels_dg_rep, 
            reels_ac_rep, playford_rep, morris_rep,
           jigs_rep, hornpipes_rep]

combined_text = ""

for rep in rep_list:
    combined_text += str(rep)


In [ ]:
# Create vocab for all repertoires
num_to_char, char_to_num = create_dictionaries(combined_text)
vocab_length = len(num_to_char)
vocab_length

### Create training data for all repertoires combined

In [ ]:
vocab_length = len(num_to_char)
num_song_list = []

# Get list of all songs in combined repertoire
song_list = []

for rep in rep_list:
    for i in rep.songs:
        song_list.append(rep.songs[i])
    
for r in rep_list:
    n_songs = len(r.songs)
    part_len_tmp = min(len(list("".join(r.songs[i].part))) for i in range(1,n_songs+1)) - 2
    k_len_tmp = min([len(list(r.songs[i].metadata['K'])) for i in range(1,n_songs+1)])
    m_len_tmp = min([len(list(r.songs[i].metadata['M'])) for i in range(1,n_songs+1)])
    
    if part_len_tmp < part_len: part_len = part_len_tmp
    if k_len_tmp < k_len: k_len = k_len_tmp
    if m_len_tmp < m_len: m_len = m_len_tmp
        
print(part_len, k_len, m_len)

In [ ]:
total_songs = len(song_list)
total_songs_train = int(0.95*total_songs)

part_num_matrix = np.zeros((total_songs_train, part_len))
y_vals = []
k_num_matrix = np.zeros((total_songs_train, k_len))
m_num_matrix = np.zeros((total_songs_train, m_len))

row_ind = 0 # row index


for i in range(total_songs_train):
    song = song_list[i]
    part_string = "".join(song.part)
    part_num_matrix[row_ind,] = np.array([char_to_num[part_string[0:part_len][j]] for j in range(part_len)])

    k_string = "".join(song.metadata['K'])
    k_num_matrix[row_ind,0] = np.array([char_to_num[k_string[j]] for j in range(k_len)])

    m_string = "".join(song.metadata['M'])
    m_num_matrix[row_ind,] = np.array([char_to_num[m_string[j]] for j in range(m_len)])

    y_vals.append(char_to_num[part_string[part_len]])

    # increase row index after every song
    row_ind += 1

# Convert y_vals to one_hot_encoded vectors
y_data = to_categorical(y_vals, num_classes=vocab_length)

In [ ]:
m_num_matrix.shape

In [ ]:
mod5 = LSTM_Builder4([200, 336, 200], 0.2, 50, 
                     256,vocab_length,
                     part_num_matrix, k_num_matrix, m_num_matrix)

mod5.summary()

In [ ]:
mod5.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# Use checkpoints to save training weights before the model finishes training
weights_filepath = "mod5_weights.hdf5"

checkpoint = ModelCheckpoint(
    weights_filepath, monitor="loss", verbose=0,
    save_best_only=True, mode="min")

callbacks_list = [checkpoint]

In [ ]:
mod5.fit([part_num_matrix, k_num_matrix, m_num_matrix],
         y_data, 
         epochs=5000, batch_size=100, shuffle=True, callbacks=callbacks_list)

### Make Predictions

In [ ]:
# Set up testing data
my_session = tf.Session()
mod5.reset_states()

starter = song_list[total_songs_train]
start_ind = total_songs_train
end_ind = len(song_list)
seq_len = 40

input_part = np.zeros((1, part_len))
sequences = input_part
input_k = np.zeros((1, k_len))
input_m = np.zeros((1, m_len))

for i in range(start_ind,start_ind+1):
    song = song_list[i]
    
    part_string = "".join(song.part)
    k_string = "".join(song.metadata['K'])
    m_string = "".join(song.metadata['M'])
    
    input_part[i-start_ind,] = np.array([char_to_num[part_string[0:part_len][j]] for j in range(part_len)])
    input_k[i-start_ind,] = np.array([char_to_num[k_string[j]] for j in range(k_len)])
    input_m[i-start_ind,] = np.array([char_to_num[m_string[j]] for j in range(m_len)])

# Starting string:
beginning = str(starter)
result = []


# Generate predictions
for i in range(seq_len):
    prediction = mod5.predict([input_part, input_k, input_m])
    
    # Sample from output of probabilities
    sample = tf.random.categorical(prediction, num_samples=1)
    sample = tf.squeeze(sample, axis=-1)
    numeric_pred = my_session.run(sample)[0]
    
    # Pass the prediction, along with the hidden state, back to the model
    sequences = np.append(sequences, numeric_pred)
    input_part = sequences[i:part_len+i].reshape(1, part_len)
    
    # Text predictions
    text_val = num_to_char[numeric_pred]
    result.append(text_val)

result = "".join(result)
print(beginning)
print("______")
print(result)

